**EQUIPO:**

*E3-CTRL Z*

- Sebastian Correa
- Miguel Jaramillo
- Luisa Castaño
- Valentina Arana

In [8]:
import pandas as pd
import numpy as np

# 1. Cargar archivo
archivo_entrada = 'Salida_sin_modificar.csv'
df = pd.read_csv(archivo_entrada)

print(f"Filas: {len(df)} | Columnas iniciales: {len(df.columns)}")

# 2. Poner nombres correctos

rescate_mongo = {
    'Pool Area_$numberLong': 'Pool Area',
    'Garage Cars_$numberLong': 'Garage Cars',
    'Garage Area_$numberLong': 'Garage Area',
    'Total Bsmt SF_$numberLong': 'Total Bsmt SF',
    'BsmtFin SF 1_$numberLong': 'BsmtFin SF 1',
    'BsmtFin SF 2_$numberLong': 'BsmtFin SF 2',
    'Bsmt Unf SF_$numberLong': 'Bsmt Unf SF',
    'Misc Val_$numberLong': 'Misc Val',
    'Bsmt Full Bath_$numberLong': 'Bsmt Full Bath',
    'Bsmt Half Bath_$numberLong': 'Bsmt Half Bath',
    'Garage Yr Blt_$numberLong': 'Garage Yr Blt'
}

for col_sucia, col_limpia in rescate_mongo.items():
    if col_sucia in df.columns:
        # Convertimos a número, los errores o nulos se vuelven 0
        datos = pd.to_numeric(df[col_sucia], errors='coerce').fillna(0)

        # Si la columna limpia ya existe
        if col_limpia in df.columns:
            df[col_limpia] = datos
            df.drop(columns=[col_sucia], inplace=True)
        else:
            # Si no existe, renombramos la sucia a limpia
            df.rename(columns={col_sucia: col_limpia}, inplace=True)

# 3. Eliminar las columnas que no van
cols_basura = ['PID JOIN', '$oid', 'PID_$numberLong', 'Order_$numberLong',
               'PID_Join1', 'PID_Join2', 'PID_mongo', 'Unnamed: 0', 'Order']
df.drop(columns=[c for c in cols_basura if c in df.columns], inplace=True, errors='ignore')

# 4. Renombrar final
rename_fix = {
    'Kitchen': 'Kitchen AbvGr',      # Validador pide Kitchen AbvGr
    'Bedroom AbvGr': 'Bedroom',      # Validador pide Bedroom
    'LotShape': 'Lot Shape',         # Espacios
    'LandContour': 'Land Contour',   # Espacios
    'GrLivArea': 'Gr Liv Area',      # Espacios
    'MSZoning': 'MS Zoning',         # Espacios
    'FullBath': 'Full Bath',
    'HalfBath': 'Half Bath',
    'MoSold': 'Mo Sold',
    'YrSold': 'Yr Sold',
    'SaleType': 'Sale Type',
    'SaleCondition': 'Sale Condition',
    'MSSubClass': 'MS SubClass',
    'ExterQual': 'Exter Qual',
    'ExterCond': 'Exter Cond',
    'HeatingQC': 'Heating QC',
    'KitchenQual': 'Kitchen Qual',
    'TotRmsAbvGrd': 'TotRms AbvGrd',
    'MasVnrArea': 'Mas Vnr Area',
    'MasVnrType': 'Mas Vnr Type',
    'CentralAir': 'Central Air',
    'RoofStyle': 'Roof Style',
    'RoofMatl': 'Roof Matl',
    'Exterior1st': 'Exterior 1st',
    'Exterior2nd': 'Exterior 2nd',
    'PavedDrive': 'Paved Drive',
    'WoodDeckSF': 'Wood Deck SF',
    'OpenPorchSF': 'Open Porch SF',
    'EnclosedPorch': 'Enclosed Porch',
    '3SsnPorch': '3Ssn Porch',
    'ScreenPorch': 'Screen Porch',
    'FireplaceQu': 'Fireplace Qu',
    '1stFlrSF': '1st Flr SF',
    '2ndFlrSF': '2nd Flr SF',
    'LowQualFinSF': 'Low Qual Fin SF'
}
df.rename(columns=rename_fix, inplace=True)

# 5. Traducción

# A. MS ZONING (Números a Códigos)
mapa_zoning = {1: 'A (agr)', 2: 'C (all)', 3: 'FV', 4: 'I (all)', 5: 'RH', 6: 'RL', 7: 'RP', 8: 'RM'}
if 'MS Zoning' in df.columns:
    df['MS Zoning'] = pd.to_numeric(df['MS Zoning'], errors='coerce')
    df['MS Zoning'] = df['MS Zoning'].map(mapa_zoning).fillna('RL')

# B. CALIDADES (Números a Letras)
mapa_calidad = {1: 'Ex', 2: 'Gd', 3: 'TA', 4: 'Fa', 5: 'Po'}
for col in ['Exter Qual', 'Exter Cond', 'Heating QC', 'Kitchen Qual']:
    if col in df.columns:
        # Si es numérico, mapeamos. Si ya es texto, forzamos reemplazo por si acaso
        df[col] = pd.to_numeric(df[col], errors='coerce').map(mapa_calidad).fillna('TA')

# C. NEIGHBORHOOD (Descripciones a Códigos)
mapa_barrios = {
    'Bloomington Heights': 'Blmngtn', 'Bluestem': 'Blueste', 'Briardale': 'BrDale',
    'Brookside': 'BrkSide', 'Clear Creek': 'ClearCr', 'College Creek': 'CollgCr',
    'Crawford': 'Crawfor', 'Edwards': 'Edwards', 'Gilbert': 'Gilbert',
    'Iowa DOT and Rail Road': 'IDOTRR', 'Meadow Village': 'MeadowV', 'Mitchell': 'Mitchel',
    'North Ames': 'Names', 'Northridge': 'NoRidge', 'Northpark Villa': 'NPkVill',
    'Northridge Heights': 'NridgHt', 'Northwest Ames': 'NWAmes', 'Old Town': 'OldTown',
    'South & West of Iowa State University': 'SWISU', 'Sawyer': 'Sawyer',
    'Sawyer West': 'SawyerW', 'Somerset': 'Somerst', 'Stone Brook': 'StoneBr',
    'Timberland': 'Timber', 'Veenker': 'Veenker', 'Greens': 'Greens',
    'GrnHill': 'GrnHill', 'Landmrk': 'Landmrk'
}
if 'Neighborhood' in df.columns:
    df['Neighborhood'] = df['Neighborhood'].replace(mapa_barrios)

# D. CONDITION 1
mapa_cond1 = {
    'Adjacent to arterial street': 'Artery', 'Adjacent to feeder street': 'Feedr',
    'Normal': 'Norm', 'Within 200\' of North-South Railroad': 'RRNn',
    'Adjacent to North-South Railroad': 'RRAn',
    'Near positive off-site feature--park greenbelt etc.': 'PosN',
    'Adjacent to postive off-site feature': 'PosA',
    'Within 200\' of East-West Railroad': 'RRNe',
    'Adjacent to East-West Railroad': 'RRAe'
}
if 'Condition 1' in df.columns:
    df['Condition 1'] = df['Condition 1'].replace(mapa_cond1)

# E. LOT SHAPE
mapa_shape = {
    'Regular': 'Reg', 'Slightly irregular': 'IR1',
    'Moderately Irregular': 'IR2', 'Irregular': 'IR3'
}
if 'Lot Shape' in df.columns:
    df['Lot Shape'] = df['Lot Shape'].replace(mapa_shape)

# 6. Rellenar

# Year Remod/Add
df['Year Remod/Add'] = pd.to_numeric(df['Year Remod/Add'], errors='coerce')
df['Year Built'] = pd.to_numeric(df['Year Built'], errors='coerce')
df['Year Remod/Add'] = df['Year Remod/Add'].fillna(df['Year Built'])

# Forzar Numéricos Enteros (Para evitar el error de Sumas y Tipos)
cols_enteros = [
    'Gr Liv Area', '1st Flr SF', '2nd Flr SF', 'Low Qual Fin SF',
    'Full Bath', 'Half Bath', 'Bedroom', 'TotRms AbvGrd',
    'Fireplaces', 'Garage Cars', 'Garage Area', 'Total Bsmt SF',
    'Year Built', 'Year Remod/Add', 'Mo Sold', 'Yr Sold', 'SalePrice',
    'Lot Area', 'Misc Val', 'Pool Area', 'Mas Vnr Area',
    'Bsmt Full Bath', 'Bsmt Half Bath', 'Lot Frontage'
]

for col in cols_enteros:
    if col in df.columns:
        df[col] = pd.to_numeric(df[col], errors='coerce').fillna(0).astype(int)

# Forzar Textos (Rellenar con NA)
cols_texto = [
    'Garage Qual', 'Garage Cond', 'Pool QC', 'Bsmt Qual', 'Bsmt Cond',
    'Alley', 'Fence', 'Fireplace Qu', 'Misc Feature', 'Mas Vnr Type',
    'Garage Type', 'Garage Finish', 'Bsmt Exposure', 'BsmtFin Type 1', 'BsmtFin Type 2'
]
for col in cols_texto:
    if col in df.columns:
        df[col] = df[col].fillna('NA').astype(str)

# 7. EXPORTAR
df.to_csv('salida.csv', index=False)

print("-" * 40)
print(f"Columnas finales: {len(df.columns)}")

Filas: 2930 | Columnas iniciales: 87
----------------------------------------
Columnas finales: 81
